In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta, date

import pandas as pd
from typing import Type
from abc import ABC, abstractmethod

from quantbullet.utils.data import generate_fake_bond_trades
from quantbullet.model_selection.rolling_window import RollingWindowManager
from quantbullet.utils.consolidator import Consolidator

from quantbullet.model import Model, Feature, FeatureSpec, FeatureRole, DataType

c:\Users\yimin\Documents\GithubCodes\quantbullet\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
class Model(ABC):
    @abstractmethod
    def fit(self, X, y):
        pass

    @abstractmethod
    def predict(self, X):
        pass

class SimpleMeanModel(Model):
    def fit(self, X, y):
        self.mean_return = y.groupby(X.index.get_level_values('security')).mean()

    def predict(self, X):
        return X.index.get_level_values('security').map(self.mean_return).values

class ResultLogger:
    def __init__(self):
        self.predictions = []

    def log(self, date, security, prediction, actual):
        self.predictions.append({
            "date": date,
            "security": security,
            "prediction": prediction,
            "actual": actual
        })

    def to_frame(self):
        return pd.DataFrame(self.predictions)

class Backtester:
    def __init__(self, data: pd.DataFrame, model_class: Type[Model], window_size: int):
        self.data_handler = DataHandler(data)
        self.model_class = model_class
        self.window_size = window_size
        self.logger = ResultLogger()

    def run(self):
        
        all_dates = sorted(self.data_handler.df.index.get_level_values('date').unique())
        for i in range(self.window_size, len(all_dates) - 1):
            current_date = all_dates[i]
            window_data = self.data_handler.get_window(current_date, self.window_size)
            next_day_data = self.data_handler.get_next_day_data(current_date)

            if next_day_data is None:
                break

            X_train = window_data[['feature']]  # change this as needed
            y_train = window_data['target']
            X_test = next_day_data[['feature']]
            y_test = next_day_data['target']

            model = self.model_class()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            for sec, pred, actual in zip(X_test.index.get_level_values('security'), y_pred, y_test):
                self.logger.log(current_date, sec, pred, actual)

        return self.logger.to_frame()


In [4]:
START_DATE = date( 2025, 1, 1 )
END_DATE = date( 2025, 4, 30 )

In [5]:
df = generate_fake_bond_trades( start_date=START_DATE, end_date=END_DATE )

In [6]:
rwm = RollingWindowManager.from_flat( df, date_col='date', window_size=5 )

In [7]:
res = rwm.get_all_window_dates()

In [ ]:
rwm.get_window_data()